In [3]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [4]:
load_dotenv()

gropApiKey = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=gropApiKey, 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


### Webscrapper from the website

In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/de/job/R-36827")
page_data = loader.load().pop().page_content
print(page_data)

Bewirb dich für Lead Software Engineer, ITC

JobsucheNavigation überspringenJobsucheNIKE, INC. JOBSBefristete JobsJoin The Talent CommunityUnser Leben bei NikeÜbersichtLeistungenMarkenÜbersichtJordanConverseTeamsÜbersichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public Affairs
Human ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyStandorteÜbersichtNike WHQNike New York HQEHQ: Hilversum, NiederlandeELC: Laakdal, BelgiumGreater China HQDiversität, Gleichstellung und InklusionÜbersichtMilitärBehinderung und InklusionInklusion der indigenen BevölkerungPraktikaTechnologyLead Software Engineer, ITCKarnataka, IndiaWerde Teil

### Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

### JSON Encoding

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
 'skills': ['Integrated Business Planning solutions like O9',
  'PL/SQL',
  'Cloud environments such as AWS, Azure or GCP',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines'],
 'description': 'Lead and mentor a team of engineers with a focus to develop solutions for the various products of Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}

In [10]:
type(json_res)

dict

#### Reading the CSV file

In [12]:
import pandas as pd

df = pd.read_csv('..\\myPortfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


### Adding the data to the database on disk

In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [15]:
job = json_res

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}]]

In [18]:
job['skills']

['Integrated Business Planning solutions like O9',
 'PL/SQL',
 'Cloud environments such as AWS, Azure or GCP',
 'Scrum and other Agile processes',
 'Jira, Git/SVN, Jenkins, CI/CD pipelines']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Solutions for Your ITC Needs

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, ITC, and I believe AtliQ can provide the expertise and support you need to develop innovative solutions for your demand and supply planning capabilities.

At AtliQ, we specialize in AI and software consulting, empowering enterprises to streamline their business processes through automated tools. Our team of experts has a proven track record of delivering tailored solutions that drive scalability, process optimization, cost reduction, and heightened overall efficiency.

Our capabilities align perfectly with your requirements, particularly in the areas of:

* Integrated Business Planning solutions
* Cloud environments such as AWS, Azure, or GCP
* DevOps practices and CI/CD pipelines
* Agile methodologies like Scrum

We have a strong portfolio of successful projects that demonstrate our expertise in these areas. Some relevant examples inc